## Skills

### Naming Scheme:

**Attack Boost 2: ATK +5**

| Attack Boost | 2 | ATK +5 |
|-|-|-|
| Common | Lvl | Descript. |

- sc_id (skill common id) : id for skill independent of level
- skill_id : id for specific skill & level
- skill_category : weapon, equipment, group, etc.
- skill_lv : level of skill


In [82]:
# head cell
import urllib.request
import json
import pandas as pd
import numpy as np
import re

### Common Skill IDs, Names, & Descriptions
#### Source
natives/STM/GameDesign/Text/Excel_Equip/SkillCommon.msg.23.json
<details>

```
"entries": [
    "name": "SkillCommon_m1961116288",
    "content": [
        "Rapid Morph",
    "name": "SkillCommon_EXPm1961116288",
    "content": [
        "Increases switch speed and power\r\nfor switch axes and charge blades.",
```
</details>

#### df_skill_common_ids
|common_id|guid_n|name|guid_d|description|
|-|-|-|-|-|
|SkillCommon_0|82d0863d-8ea5-4106-9392-a8b7575f6b9f|SkillCommon_0|ab11e7f5-8b70-4dff-a0ac-f9643ef55e6a|SkillCommon_EXP0|
|SkillCommon_1|c5243424-047c-40cf-a6f4-69e9d3ae860f|Attack Boost|38b4b21e-d241-4c08-ae35-cff0632574e2|Increases attack power.|
|SkillCommon_m2096489472|c4dc2439-1d42-4ef4-bca9-07be35f4bebd|Critical Eye|768c9e10-dfe1-4373-937c-d6376e92a26a|Increases affinity.|
|SkillCommon_m1607763456|6e7e9e83-16b1-4bf6-92e9-98eff98d85ac|Critical Boost|671c0922-1fe5-4c78-aac4-213ea807daa1|Increases the damage of critical hits.|
|SkillCommon_313598432|b09b9e70-7243-45c5-86e3-158ed24004e8|Critical Element|f1506057-3237-400b-a7aa-4f6619236f11|Increases elemental damage (fire, water, thunder, ice, dragon) when landing critical hits.|

#### Dicts
- dict_common_guid_names { guid_n : name }
- dict_common_guid_desc { guid_d : description }
- dict_common_id_names { sc_id : name }

In [83]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/SkillCommon.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1.rename(columns={'name':'id','content':'name', 'guid':'guid_n'}, inplace=True)
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description', 'guid':'guid_d'}, inplace=True)

#adjust content
df_1['name'] = df_1['name'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)
df_2['description'] = df_2['description'].str.replace('<BOLD>','')
df_2['description'] = df_2['description'].str.replace('</BOLD>','')

#merge into final
df = pd.merge(df_1[['id','guid_n','name']], df_2[['id','guid_d','description']], on='id', how='left')
df.reset_index(drop=True, inplace=True)

#dicts
dict_common_guid_names = dict(zip(df['guid_n'], df['name']))
dict_common_guid_desc = dict(zip(df['guid_d'], df['description']))
dict_common_id_names = dict(zip(df['id'], df['name']))

#format skill common id
# df['id'] = df['id'].str.replace('SkillCommon_','')
# df['id'] = df['id'].str.replace('m','-')
# df.rename(columns={'id':'skill_common_id'}, inplace=True)

df.rename(columns={'id':'common_id'}, inplace=True)

df_skill_common_ids = df.copy()

df_skill_common_ids.head()
# df_skill_common.to_excel('text skill common.xlsx')

,common_id,guid_n,name,guid_d,description
0,SkillCommon_0,82d0863d-8ea5-4106-9392-a8b7575f6b9f,SkillCommon_0,ab11e7f5-8b70-4dff-a0ac-f9643ef55e6a,SkillCommon_EXP0
1,SkillCommon_1,c5243424-047c-40cf-a6f4-69e9d3ae860f,Attack Boost,38b4b21e-d241-4c08-ae35-cff0632574e2,Increases attack power.
2,SkillCommon_m2096489472,c4dc2439-1d42-4ef4-bca9-07be35f4bebd,Critical Eye,768c9e10-dfe1-4373-937c-d6376e92a26a,Increases affinity.
3,SkillCommon_m1607763456,6e7e9e83-16b1-4bf6-92e9-98eff98d85ac,Critical Boost,671c0922-1fe5-4c78-aac4-213ea807daa1,Increases the damage of critical hits.
4,SkillCommon_313598432,b09b9e70-7243-45c5-86e3-158ed24004e8,Critical Element,f1506057-3237-400b-a7aa-4f6619236f11,"Increases elemental damage (fire, water, thund..."


### Specific Skill IDs & Descriptions

#### Source
natives/STM/GameDesign/Text/Excel_Equip/Skill.msg.23.json
<details>

```
"entries": [
    {
        "name": "Skill_488",
      "guid": "95f8239f-d0c1-4120-b3fb-37378e8875f5",
      ...
      "content": [
        "<COLOR FF0000>#Rejected#</COLOR> Skill_488",
        "<COLOR FF0000>#Rejected#</COLOR> Skill_488",]
    },
    {
      "name": "Skill_EXP488",
      "guid": "f72a7bc8-58c8-4550-afc3-2450a5b463c2",
      ...
      "content": [
        "会心攻撃時のダメージ倍率を\r\n１．３７倍に強化する",
        "Increases damage dealt\r\nby critical hits to 37%.",
      ]
    },
```
</details>

#### df_skill_spec_ids
|spec_id|guid_n|name|guid_d|description|
|-|-|-|-|-|
|Skill_1|f7457609-c698-44dc-a87c-3013a81fe34f|Skill_1|3b08702a-e4a1-4de4-b7db-f5897fef1c4b|Skill_EXP1|
|Skill_2|824fa7e2-6344-4f5d-a140-0d411ccc674d|Skill_2|96e65c81-c4a1-4fb0-aea4-c256200cda88|Attack +3|
|Skill_3|c3858b4a-0f53-4739-ade5-ddf1c08497ad|Skill_3|b97ead47-2c69-436f-ab9d-4f9059ec151f|Attack +5|
|Skill_4|2a36d1c4-d599-4b2d-8385-47ebef1df235|Skill_4|9d878426-10dd-4506-9259-6548d4c59735|Attack +7|
|Skill_5|85873022-2c20-4f4a-9e13-6c2e1275918a|Skill_5|86a03b9f-c136-414c-8fe5-501b6c4b6b73|Attack +2% Bonus: +8|

#### Dicts
- dict_spec_guid_names: { guid_n : name }
- dict_spec_guid_desc: { guid_d : description }

In [84]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/Skill.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1.rename(columns={'name':'id','content':'name', 'guid':'guid_n'}, inplace=True)
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description', 'guid':'guid_d'}, inplace=True)

#adjust content
# df_1 = df_1[~df_1['name'].str.contains('<COLOR FF0000>')].copy()
# df_2 = df_2[~df_2['description'].str.contains('<COLOR FF0000>')].copy()
df_1['name'] = df_1['name'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)
df_2['description'] = df_2['description'].str.replace('<BOLD>','')
df_2['description'] = df_2['description'].str.replace('</BOLD>','')


#merge into final
df = pd.merge(df_1[['id','guid_n','name']], df_2[['id','guid_d','description']], on='id', how='left')
df.reset_index(drop=True, inplace=True)

#dicts
dict_spec_guid_names = dict(zip(df['guid_n'], df['name']))
dict_spec_guid_desc = dict(zip(df['guid_d'], df['description']))

#format skill ids
# df['id'] = df['id'].str.replace('Skill_','')
df.rename(columns={'id':'spec_id'}, inplace=True)

df_skill_spec_ids = df.copy()

# df_1.head()
df_skill_spec_ids.head()
# df_skills.to_excel('text skill.xlsx')

,spec_id,guid_n,name,guid_d,description
0,Skill_1,f7457609-c698-44dc-a87c-3013a81fe34f,Skill_1,3b08702a-e4a1-4de4-b7db-f5897fef1c4b,Skill_EXP1
1,Skill_2,824fa7e2-6344-4f5d-a140-0d411ccc674d,Skill_2,96e65c81-c4a1-4fb0-aea4-c256200cda88,Attack +3
2,Skill_3,c3858b4a-0f53-4739-ade5-ddf1c08497ad,Skill_3,b97ead47-2c69-436f-ab9d-4f9059ec151f,Attack +5
3,Skill_4,2a36d1c4-d599-4b2d-8385-47ebef1df235,Skill_4,9d878426-10dd-4506-9259-6548d4c59735,Attack +7
4,Skill_5,85873022-2c20-4f4a-9e13-6c2e1275918a,Skill_5,86a03b9f-c136-414c-8fe5-501b6c4b6b73,Attack +2% Bonus: +8


### Common Skills

#### Source
natives/STM/GameDesign/Common/Equip/SkillCommonData.user.3.json
<details>

```
"app.user_data.SkillCommonData": {
    "_Values": [
        {
            "app.user_data.SkillCommonData.cData": {
                "_Index": 4,
                "_skillId": "[-181127504]HunterSkill_028",
                "_skillType": "[441208832]NONE",
                "_skillCategory": "[3]WEAPON",
                "_SkillIconType": "[1]SKILL_0000",
                "_skillName": "b759c7ca-775e-4779-ab53-f94bf2962305",
                "_skillExplain": "7c1468b7-0445-4dc1-acb2-6d5bf228f9f5",
                "_SortId": 4
            }
        },
```
</details>

#### df_common_skills
|name|category|description|SkillIconType|common_hunter_id|common_id|hunter_id|SortId|
|-|-|-|-|-|-|-|-|
|SkillCommon_0|[0]EQUIP|SkillCommon_EXP0|[1]SKILL_0000|[0]NONE|SkillCommon_0|NONE|0|
|Attack Boost|[3]WEAPON|Increases attack power.|[1]SKILL_0000|[1]HunterSkill_000|SkillCommon_1|HunterSkill_000|1|
|Resentment|[0]EQUIP|Increases attack when you have recoverable damage (the red portion of your Health Gauge).|[1]SKILL_0000|[1359821952]HunterSkill_060|SkillCommon_1359821952|HunterSkill_060|2|
|Adrenaline Rush|[0]EQUIP|Temporarily increases attack power after executing a perfectly-timed evade just as a monster attacks.|[1]SKILL_0000|[1174975744]HunterSkill_115|SkillCommon_1174975744|HunterSkill_115|3|
|Offensive Guard|[3]WEAPON|Temporarily increases attack power after executing a perfectly-timed guard.|[1]SKILL_0000|[-181127504]HunterSkill_028|SkillCommon_m181127504|HunterSkill_028|4|

#### Dicts
- dict_common_hunter_name: { common_hunter_id : name }

In [85]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/SkillCommonData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
df = pd.json_normalize(raw, record_path=['app.user_data.SkillCommonData', '_Values'])
df.columns = df.columns.str.replace('app.user_data.SkillCommonData.cData._','')

#split ids
# df['hs_id'] = df['skillId'].str.extract(r'^(?:.*_)(\d+)$')
df['hunter_id'] = df['skillId'].str.extract(r'^\[[^]]+\](.*)$')
df['common_id'] = 'SkillCommon_' + df['skillId'].str.extract(r'\[(.*?)\]')
df['common_id'] = df['common_id'].str.replace('-','m')

df['skillName'] = df['skillName'].replace(dict_common_guid_names)
df['skillExplain'] = df['skillExplain'].replace(dict_common_guid_desc)

df_common_skills = df[['skillName','skillCategory','skillExplain','SkillIconType','skillId','common_id','hunter_id','SortId']].copy()
df.reset_index(drop=True, inplace=True)

df_common_skills.rename(columns={'skillName':'name',
                   'skillExplain':'description',
                   'skillCategory':'category',
                   'skillId':'common_hunter_id'}, inplace=True)

#dicts
dict_common_hunter_name = dict(zip(df_common_skills['common_hunter_id'],df_common_skills['name']))

df_common_skills.head()
# df_common_skills.to_excel('common_skills.xlsx')

,name,category,description,SkillIconType,common_hunter_id,common_id,hunter_id,SortId
0,SkillCommon_0,[0]EQUIP,SkillCommon_EXP0,[1]SKILL_0000,[0]NONE,SkillCommon_0,NONE,0
1,Attack Boost,[3]WEAPON,Increases attack power.,[1]SKILL_0000,[1]HunterSkill_000,SkillCommon_1,HunterSkill_000,1
2,Resentment,[0]EQUIP,Increases attack when you have recoverable dam...,[1]SKILL_0000,[1359821952]HunterSkill_060,SkillCommon_1359821952,HunterSkill_060,2
3,Adrenaline Rush,[0]EQUIP,Temporarily increases attack power after execu...,[1]SKILL_0000,[1174975744]HunterSkill_115,SkillCommon_1174975744,HunterSkill_115,3
4,Offensive Guard,[3]WEAPON,Temporarily increases attack power after execu...,[1]SKILL_0000,[-181127504]HunterSkill_028,SkillCommon_m181127504,HunterSkill_028,4


### Specific Skill Detail Relations
#### Source
natives/STM/GameDesign/Common/Equip/SkillData.user.3.json
<details>

```
"app.user_data.SkillData": {
    "_Values": [
        {
            "app.user_data.SkillData.cData": {
                "_Index": 3,
                "_dataId": 4,
                "_skillId": "[1]HunterSkill_000",
                "_SkillLv": 3,
                "_skillName": "2a36d1c4-d599-4b2d-8385-47ebef1df235",
                "_skillExplain": "9d878426-10dd-4506-9259-6548d4c59735",
                "_openSkill": [
                    "[1]HunterSkill_000",
                    "[0]NONE",
                    "[0]NONE",
                    "[0]NONE",
                    "[0]NONE"
                ],
                "_value": [
                    100,
                    7,
                    0,
                    0
                ]
            }
        },
```
</details>

#### df_spec_skills
|Index|spec_id|common_name|SkillLv|spec_name|description|value|os1|os2|os3|os4|os5|
|-|-|-|-|-|-|-|-|-|-|-|-|
|0|Skill_1|SkillCommon_0|0|Skill_1|Skill_EXP1|[0, 0, 0, 0]|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|1|Skill_2|Attack Boost|1|Attack Boost 1|Attack +3|[100, 3, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|2|Skill_3|Attack Boost|2|Attack Boost 2|Attack +5|[100, 5, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|3|Skill_4|Attack Boost|3|Attack Boost 3|Attack +7|[100, 7, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|
|4|Skill_5|Attack Boost|4|Attack Boost 4|Attack +2% Bonus: +8|[102, 8, 0, 0]|Attack Boost|SkillCommon_0|SkillCommon_0|SkillCommon_0|SkillCommon_0|


#### Dicts

- dict_backfill = dict(zip(df_1['os1'],df_1['spec_name']))

In [86]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/SkillData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
df = pd.json_normalize(raw, record_path=['app.user_data.SkillData', '_Values'])
df.columns = df.columns.str.replace('app.user_data.SkillData.cData._','')

#split openSkill
df[['os1','os2','os3','os4','os5']] = pd.DataFrame(df['openSkill'].tolist(), index=df.index)

#replace os data
df[['os1','os2','os3','os4','os5']] = df[['os1','os2','os3','os4','os5']].replace(dict_common_hunter_name)

#recombine
df = df.drop('openSkill', axis=1)

#data replacement
df['skillName'] = df['skillName'].replace(dict_spec_guid_names)
df['skillExplain'] = df['skillExplain'].replace(dict_spec_guid_desc)
df['skillId'] = df['skillId'].replace(dict_common_hunter_name)

# mask = df['os1'].str.contains('_')
# df_1 = df.loc[mask]
# dict_backfill = dict(zip(df_1['os1'],df_1['spec_name']))

# specific skill name fill blanks with generics
mask = df['skillName'].str.contains('Skill_') & ~df['skillId'].str.contains('SkillCommon_')
df.loc[mask, 'skillName'] = df.loc[mask, 'skillId'] + ' ' + df.loc[mask, 'SkillLv'].astype(str)

#adjust specific skill id
df['dataId'] = 'Skill_' + df['dataId'].astype(str)

df.rename(columns={'dataId':'spec_id',
                   'skillId':'common_name',
                   'skillExplain':'description',
                   'skillName':'spec_name'}, inplace=True)
df_spec_skills = df.copy()
df_spec_skills.reset_index(drop=True, inplace=True)

df_spec_skills.head()
# df_spec_skills.to_excel('specific skills.xlsx')

,Index,spec_id,common_name,SkillLv,spec_name,description,value,os1,os2,os3,os4,os5
0,0,Skill_1,SkillCommon_0,0,Skill_1,Skill_EXP1,"[0, 0, 0, 0]",SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
1,1,Skill_2,Attack Boost,1,Attack Boost 1,Attack +3,"[100, 3, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
2,2,Skill_3,Attack Boost,2,Attack Boost 2,Attack +5,"[100, 5, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
3,3,Skill_4,Attack Boost,3,Attack Boost 3,Attack +7,"[100, 7, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0
4,4,Skill_5,Attack Boost,4,Attack Boost 4,Attack +2% Bonus: +8,"[102, 8, 0, 0]",Attack Boost,SkillCommon_0,SkillCommon_0,SkillCommon_0,SkillCommon_0


### Consolidation
#### Backfill common name data

In [87]:
# common names
mask = df_spec_skills['os1'].str.contains('SkillCommon_')
df = df_spec_skills.loc[mask]
dict_ = dict(zip(df['os1'],df['spec_name']))

df_skill_common_ids['name'] = df_skill_common_ids['name'].replace(dict_)
df_skill_common_ids['name'] = df_skill_common_ids['name'].str.replace(' II','')

# forwardfill common skills
dict_ = dict(zip(df_skill_common_ids['common_id'],df_skill_common_ids['name']))
df_common_skills['name'] = df_common_skills['name'].replace(dict_)

# HunterSkill recover
mask = df_spec_skills['os1'].str.contains('HunterSkill_')
df = df_spec_skills.loc[mask]
df = df.iloc[1:]
df = df[['spec_name','os1']]
df.columns = ['name','common_hunter_id']
df['hunter_id'] = df['common_hunter_id'].str.extract(r'^\[[^]]+\](.*)$')
df['common_id'] = 'SkillCommon_' + df['common_hunter_id'].str.extract(r'\[(.*?)\]')
df['common_id'] = df['common_id'].str.replace('-','m')
df['description'] = 'SkillCommon_EXP' + df['common_hunter_id'].str.extract(r'\[(.*?)\]')
df['description'] = df['description'].str.replace('-','m')
df_common_skills = pd.concat([df_common_skills,df], ignore_index=True)
df_common_skills['name'] = df_common_skills['name'].str.replace(' II','')


# df_skill_common_ids
df_common_skills.head()
# df_common_skills.to_excel('common skill.xlsx')

,name,category,description,SkillIconType,common_hunter_id,common_id,hunter_id,SortId
0,Skill_1,[0]EQUIP,SkillCommon_EXP0,[1]SKILL_0000,[0]NONE,SkillCommon_0,NONE,0.0
1,Attack Boost,[3]WEAPON,Increases attack power.,[1]SKILL_0000,[1]HunterSkill_000,SkillCommon_1,HunterSkill_000,1.0
2,Resentment,[0]EQUIP,Increases attack when you have recoverable dam...,[1]SKILL_0000,[1359821952]HunterSkill_060,SkillCommon_1359821952,HunterSkill_060,2.0
3,Adrenaline Rush,[0]EQUIP,Temporarily increases attack power after execu...,[1]SKILL_0000,[1174975744]HunterSkill_115,SkillCommon_1174975744,HunterSkill_115,3.0
4,Offensive Guard,[3]WEAPON,Temporarily increases attack power after execu...,[1]SKILL_0000,[-181127504]HunterSkill_028,SkillCommon_m181127504,HunterSkill_028,4.0


#### Backfill specific name data

In [88]:
dict_ = dict(zip(df_spec_skills['spec_id'],df_spec_skills['spec_name']))
df_skill_spec_ids['name'] = df_skill_spec_ids['name'].replace(dict_)

df_skill_spec_ids.head()

,spec_id,guid_n,name,guid_d,description
0,Skill_1,f7457609-c698-44dc-a87c-3013a81fe34f,Skill_1,3b08702a-e4a1-4de4-b7db-f5897fef1c4b,Skill_EXP1
1,Skill_2,824fa7e2-6344-4f5d-a140-0d411ccc674d,Attack Boost 1,96e65c81-c4a1-4fb0-aea4-c256200cda88,Attack +3
2,Skill_3,c3858b4a-0f53-4739-ade5-ddf1c08497ad,Attack Boost 2,b97ead47-2c69-436f-ab9d-4f9059ec151f,Attack +5
3,Skill_4,2a36d1c4-d599-4b2d-8385-47ebef1df235,Attack Boost 3,9d878426-10dd-4506-9259-6548d4c59735,Attack +7
4,Skill_5,85873022-2c20-4f4a-9e13-6c2e1275918a,Attack Boost 4,86a03b9f-c136-414c-8fe5-501b6c4b6b73,Attack +2% Bonus: +8


#### milestone output

In [89]:
# df_common_skills.to_excel('common skill.xlsx')
# df_spec_skills[['spec_id','common_name','SkillLv','spec_name','description']].to_excel('specific skills.xlsx')

## Armors

In [90]:
# head cell
import json
import pandas as pd
import numpy as np
import re
import urllib.request

### Armor IDs
natives/STM/GameDesign/Text/Excel_Equip/Armor.msg.23.json
<details>

```
"entries": [
    "name": "Armor_ID00117",
    "content": [
        "G. Ebony Helm",
    "name": "Armor_EXPID00117",
    "content": [
        "Head armor made from Guardian\r\nEbony Odogaron spoils. To don it\r\nis to be seized by unspeakable\r\ncravings.",
```
</details>

#### df

In [91]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/Armor.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1.rename(columns={'name':'id','content':'name', 'guid':'guid_n'}, inplace=True)
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description', 'guid':'guid_d'}, inplace=True)

#adjust content
df_1['name'] = df_1['name'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace('<COLOR FF0000>#Rejected#</COLOR> ','')
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)
df_2['description'] = df_2['description'].str.replace('<BOLD>','')
df_2['description'] = df_2['description'].str.replace('</BOLD>','')

#merge into final
df = pd.merge(df_1[['id','guid_n','name']], df_2[['id','guid_d','description']], on='id', how='left')
df.reset_index(drop=True, inplace=True)

#dicts
# dict_common_guid_names = dict(zip(df['guid_n'], df['name']))
# dict_common_guid_desc = dict(zip(df['guid_d'], df['description']))
# dict_common_id_names = dict(zip(df['id'], df['name']))

df.rename(columns={'id':'armor_id'}, inplace=True)

df_armor_ids = df.copy()

df_armor_ids.head()

,armor_id,guid_n,name,guid_d,description
0,Armor_ID00001,5383ad5c-ca4d-4faa-8583-da37b738b483,Hope Mask,e54ca422-c8ef-4c71-bd0e-3e5ffbbe3c34,Armor crafted specifically for members of the ...
1,Armor_ID00002,8a8eaf84-d6f0-4983-9a1b-a6e1455116bf,Hope Mail,1f55f364-cb30-4d2b-a66a-2e4b17cf9c7b,Armor crafted specifically for members of the ...
2,Armor_ID00003,8b2913a9-14b2-4dc7-a1f1-933237a9c633,Hope Vambraces,d2bb221d-8d76-4d46-82a2-d202d699bc56,Armor crafted specifically for members of the ...
3,Armor_ID00004,343b29da-c830-4514-8395-bf72db6ec994,Hope Coil,72f93eec-b3d7-45cd-b9ed-353f4e9c9490,Armor crafted specifically for members of the ...
4,Armor_ID00005,beb67c1e-8d41-43cc-881e-3a4157be0421,Hope Greaves,cd257a97-c8f2-40b4-8049-9de1293fa3ed,Armor crafted specifically for members of the ...


### Armor Series
natives/STM/GameDesign/Text/Excel_Equip/ArmorSeries.msg.23.json
<details>

```
"entries": [
    "name": "ArmorSeries_12",
    "content": [
        "Bulaqchi",
```
</details>

In [92]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/ArmorSeries.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

df.rename(columns={'name':'series_id',
                   'content':'name'}, inplace=True)

df_armor_series = df[['series_id','guid','name']].copy()

df_armor_series.head()

,series_id,guid,name
0,ArmorSeries_0,ddac217a-52a5-40b3-814c-a310e137df9b,-
1,ArmorSeries_2,dd658f22-a56c-4128-b500-0fd7b9fb5dd0,Hope
2,ArmorSeries_3,ca4f1245-b46f-4d91-b091-c967f4069a61,Leather
3,ArmorSeries_4,b39c4924-3921-4fbd-a468-0f69540e9020,Chainmail
4,ArmorSeries_5,f23c8723-53cf-4ee3-8f7c-cbda56a555af,Bone


### Armor Data

#### Source
natives/STM/GameDesign/Common/Equip/ArmorData.user.3.json
<details>

```
"app.user_data.ArmorData": {
    "_Values": [
    {
        "app.user_data.ArmorData.cData": {
            "_Index": 11,
            "_DataValue": 12,
            "_Series": {
                "app.ArmorDef.SERIES_Serializable": {
                    "_Value": "[4]ID_003"
                }
            },
            "_PartsType": {
                "app.ArmorDef.ARMOR_PARTS_Serializable": {
                    "_Value": "[1]BODY"
                }
            },
            "_Name": "efcf7b75-8990-45d8-afff-9175814887d2",
            "_Explain": "a476e4ab-9039-4488-8637-8b2d71e0b549",
            "_Defense": 2,
            "_Resistance": [
                0,
                2,
                0,
                0,
                0
            ],
            "_SlotLevel": [
                {
                    "app.EquipDef.SlotLevel_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.EquipDef.SlotLevel_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.EquipDef.SlotLevel_Serializable": {
                        "_Value": "[0]NONE"
                    }
                }
            ],
            "_Skill": [
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[595870656]HunterSkill_090"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                },
                {
                    "app.HunterDef.Skill_Serializable": {
                        "_Value": "[0]NONE"
                    }
                }
            ],
            "_SkillLevel": [
                0,
                0,
                1,
                0,
                0,
                0,
                0
            ]
        }
    },
```
</details>

#### df_armor
|series_combo_id|series_name|part|name|Defense|Fire|Water|Thunder|Ice|Dragon|slot_1_lv|slot_2_lv|slot_3_lv|series_skill|group_skill|skill_1|skill_1_lv|skill_2|skill_2_lv|skill_3|skill_3_lv|skill_4|skill_4_lv|skill_5|skill_5_lv|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|[2]ID_001|Hope|[0]HELM|Hope Mask|2|1|0|1|0|0|0|0|0|Skill_1|Skill_1|Stun Resistance|1|Skill_1|0|Skill_1|0|Skill_1|0|Skill_1|0|
|[2]ID_001|Hope|[1]BODY|Hope Mail|2|1|0|1|0|0|0|0|0|Skill_1|Skill_1|Divine Blessing|1|Skill_1|0|Skill_1|0|Skill_1|0|Skill_1|0|
|[2]ID_001|Hope|[2]ARM|Hope Vambraces|2|1|0|1|0|0|0|0|0|Skill_1|Skill_1|Divine Blessing|1|Skill_1|0|Skill_1|0|Skill_1|0|Skill_1|0|
|[2]ID_001|Hope|[3]WAIST|Hope Coil|2|1|0|1|0|0|0|0|0|Skill_1|Skill_1|Stun Resistance|1|Skill_1|0|Skill_1|0|Skill_1|0|Skill_1|0|
|[2]ID_001|Hope|[4]LEG|Hope Greaves|2|1|0|1|0|0|0|0|0|Skill_1|Skill_1|Divine Blessing|1|Skill_1|0|Skill_1|0|Skill_1|0|Skill_1|0|


In [93]:
# Armor
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/ArmorData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw[0].get('app.user_data.ArmorData').get('_Values')
df = pd.json_normalize(sect)
df.columns = df.columns.str.replace('app.user_data.ArmorData.cData._','')

# working rename
df.rename(columns={'Series.app.ArmorDef.SERIES_Serializable._Value':'series_combo_id',
                   'PartsType.app.ArmorDef.ARMOR_PARTS_Serializable._Value':'part'}, inplace=True)
# df['Series'] = df['Series.app.ArmorDef.SERIES_Serializable._Value'].str.extract(r'\](.*)', expand=False)
# df['Part'] = df['PartsType.app.ArmorDef.ARMOR_PARTS_Serializable._Value'].str.extract(r'\](.*)', expand=False)

# resistances
df[['Fire','Water','Thunder','Ice','Dragon']] = pd.DataFrame(df['Resistance'].tolist(), index=df.index)
df = df.drop('Resistance', axis=1)

# slot levels
df['slot_1_lv'] = df['SlotLevel'].str[0].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
df['slot_2_lv'] = df['SlotLevel'].str[1].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
df['slot_3_lv'] = df['SlotLevel'].str[2].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
df = df.drop('SlotLevel', axis=1)

# slot level format
df['slot_1_lv'] = df['slot_1_lv'].str.extract(r'\[(.*?)\]').astype(int)
df['slot_2_lv'] = df['slot_2_lv'].str.extract(r'\[(.*?)\]').astype(int)
df['slot_3_lv'] = df['slot_3_lv'].str.extract(r'\[(.*?)\]').astype(int)

# skills
df['series_skill'] = df['Skill'].str[0].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['group_skill'] = df['Skill'].str[1].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_1'] = df['Skill'].str[2].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_2'] = df['Skill'].str[3].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_3'] = df['Skill'].str[4].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_4'] = df['Skill'].str[5].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_5'] = df['Skill'].str[6].str['app.HunterDef.Skill_Serializable'].str['_Value']
df = df.drop('Skill', axis=1)

# skill levels
df[['series_skill_lv','group_skill_lv','skill_1_lv','skill_2_lv','skill_3_lv','skill_4_lv','skill_5_lv']] = pd.DataFrame(df['SkillLevel'].tolist(), index=df.index)
df = df.drop('SkillLevel', axis=1)

# skill replace
dict_ = dict(zip(df_common_skills['common_hunter_id'], df_common_skills['name']))
df[['series_skill','group_skill','skill_1','skill_2','skill_3','skill_4','skill_5']] = df[['series_skill','group_skill','skill_1','skill_2','skill_3','skill_4','skill_5']].replace(dict_)

#series_id & series_id_alt split
# df['series_id'] = 'ArmorSeries_' + df['series_combo_id'].str.extract(r'\[(.*?)\]')
# df['series_id'] = df['series_id'].str.replace('-','m')
# df['series_alt_id'] = df['series_combo_id'].str.extract(r'^\[[^]]+\](.*)$')

# series_combo_id to series_name
df['series_name'] = 'ArmorSeries_' + df['series_combo_id'].str.extract(r'\[(.*?)\]')
df['series_name'] = df['series_name'].str.replace('-','m')
dict_ = dict(zip(df_armor_series['series_id'],df_armor_series['name']))
df['series_name'] = df['series_name'].replace(dict_)

# name and description replace
dict_ = dict(zip(df_armor_ids['guid_n'], df_armor_ids['name']))
df['Name'] = df['Name'].replace(dict_)
dict_ = dict(zip(df_armor_ids['guid_d'], df_armor_ids['description']))
df['Explain'] = df['Explain'].replace(dict_)

# column rename
df.rename(columns={'Name':'name',
                   'Explain':'description'},inplace=True)

# reorder and remove columns
df_armor=df[['series_combo_id','series_name','part','name','Defense','Fire','Water','Thunder','Ice','Dragon','slot_1_lv','slot_2_lv','slot_3_lv',
             'series_skill','group_skill','skill_1','skill_1_lv','skill_2','skill_2_lv','skill_3','skill_3_lv','skill_4','skill_4_lv','skill_5','skill_5_lv']].copy()
df_armor.reset_index(drop=True, inplace=True)

df_armor.head()
# df_armor.to_excel("wilds_armor.xlsx")

,series_combo_id,series_name,part,name,Defense,Fire,Water,Thunder,Ice,Dragon,...,skill_1,skill_1_lv,skill_2,skill_2_lv,skill_3,skill_3_lv,skill_4,skill_4_lv,skill_5,skill_5_lv
0,[2]ID_001,Hope,[0]HELM,Hope Mask,2,1,0,1,0,0,...,Stun Resistance,1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0
1,[2]ID_001,Hope,[1]BODY,Hope Mail,2,1,0,1,0,0,...,Divine Blessing,1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0
2,[2]ID_001,Hope,[2]ARM,Hope Vambraces,2,1,0,1,0,0,...,Divine Blessing,1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0
3,[2]ID_001,Hope,[3]WAIST,Hope Coil,2,1,0,1,0,0,...,Stun Resistance,1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0
4,[2]ID_001,Hope,[4]LEG,Hope Greaves,2,1,0,1,0,0,...,Divine Blessing,1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0


### Backfill Armor Series

In [94]:
df_1 = df_armor[['series_combo_id']].copy()
df_1['series_id'] = 'ArmorSeries_' + df_1['series_combo_id'].str.extract(r'\[(.*?)\]')
df_1['series_id'] = df_1['series_id'].str.replace('-','m')
df_1['series_alt_id'] = df_1['series_combo_id'].str.extract(r'^\[[^]]+\](.*)$')
df_1 = df_1.drop_duplicates(subset='series_id')

df_armor_series = pd.merge(df_armor_series, df_1, on='series_id', how='left')
df_armor_series.reset_index(drop=True, inplace=True)

df_armor_series.head()

,series_id,guid,name,series_combo_id,series_alt_id
0,ArmorSeries_0,ddac217a-52a5-40b3-814c-a310e137df9b,-,NaN,NaN
1,ArmorSeries_2,dd658f22-a56c-4128-b500-0fd7b9fb5dd0,Hope,[2]ID_001,ID_001
2,ArmorSeries_3,ca4f1245-b46f-4d91-b091-c967f4069a61,Leather,[3]ID_002,ID_002
3,ArmorSeries_4,b39c4924-3921-4fbd-a468-0f69540e9020,Chainmail,[4]ID_003,ID_003
4,ArmorSeries_5,f23c8723-53cf-4ee3-8f7c-cbda56a555af,Bone,[5]ID_004,ID_004


### Armor Series Other
#### Source
natives/STM/GameDesign/Common/Equip/ArmorSeriesData.user.3.json
<details>

```
"app.user_data.ArmorSeriesData": {
    "_Values": [
        {
            "app.user_data.ArmorSeriesData.cData": {
                "_Index": 36,
                "_Series": {
                    "app.ArmorDef.SERIES_Serializable": {
                        "_Value": "[-1830990336]ID_061"
                    }
                },
                "_Name": "daf67436-95a6-4d7f-8879-30070c46bdf8",
                "_ModelVariety": {
                    "app.ArmorDef.MODEL_VARIETY_Serializable": {
                        "_Value": "[0]BOTH"
                    }
                },
                "_ModId": 4,
                "_ModSubMaleId": 0,
                "_ModSubFemaleId": 1,
                "_SortId": 36,
                "_Rare": {
                    "app.ItemDef.RARE_Serializable": {
                        "_Value": "[14]RARE4"
                    }
                },
                "_Price": 2500,
                "_Color": {
                    "app.ArmorDef.ARMOR_COLOR_TYPE_Serializable": {
                        "_Value": "[-993752576]Type02"
                    }
                },
                "_IsOneSet": false
            }
        },
```
</details>

#### df -

## Decos
### Deco IDs
#### Source
natives/STM/GameDesign/Text/Excel_Equip/Accessory.msg.23.json
<details>

```
"entries": [
    "name": "Accessory_ACC_1179637760",
    "content": [
        "Gambit Jewel II [2]",
    "name": "Accessory_ACC_EXP_19401228",
    "content": [
        "A decoration that grants\r\nthe Punishing Draw skill.",
```
</details>

#### df_deco_ids
|id|name|description|
|-|-|-|
|Accessory_ACC_130724024|Attack Jewel [1]|A decoration that grants the Attack Boost skill.|
|Accessory_ACC_m1843593600|Attack Jewel II [2]|A decoration that grants the Attack Boost skill.|
|Accessory_ACC_280205024|Attack Jewel III [3]|A decoration that grants the Attack Boost skill.|
|Accessory_ACC_m451614112|Expert Jewel [1]|A decoration that grants the Critical Eye skill.|
|Accessory_ACC_248848256|Expert Jewel II [2]|A decoration that grants the Critical Eye skill.|


In [95]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/Accessory.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_2['name'] = df_2['name'].str.replace('EXP_','')
df_2.rename(columns={'name':'id','content':'description','guid':'guid_d'}, inplace=True)
df_1.rename(columns={'name':'id','content':'name','guid':'guid_n'}, inplace=True)

#adjust descriptions
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)

#merge into final
df_deco_ids = pd.merge(df_1[['id','name','guid_n']], df_2[['id','description','guid_d']], on='id', how='left')
df_deco_ids.reset_index(drop=True, inplace=True)

df_deco_ids.head()
# df_deco_ids.to_excel('deco.xlsx')

,id,name,guid_n,description,guid_d
0,Accessory_ACC_130724024,Attack Jewel [1],56bff7fd-6ef8-4038-9ce7-006a0fdc88a1,A decoration that grants the Attack Boost skill.,abb148ea-836c-4a2a-9cb8-5b74ab3c845c
1,Accessory_ACC_m1843593600,Attack Jewel II [2],f559d3b8-44ba-44c7-8b0d-68622cb42e5e,A decoration that grants the Attack Boost skill.,e8c23984-96cb-4572-98a0-25103e2f05f8
2,Accessory_ACC_280205024,Attack Jewel III [3],40d8ce15-1a4c-4b0a-837d-9dbaf1a5b80f,A decoration that grants the Attack Boost skill.,4a37c8c6-2b2c-459c-a2a0-012b2aaa94fd
3,Accessory_ACC_m451614112,Expert Jewel [1],e9b426cb-e53d-47fe-989f-81c4870d21de,A decoration that grants the Critical Eye skill.,da14a34b-9a23-4235-8d04-36fdb337f739
4,Accessory_ACC_248848256,Expert Jewel II [2],acf0fecc-b210-4f88-b359-86d2390d5bbb,A decoration that grants the Critical Eye skill.,b572c525-8a62-4158-bc98-0b9fdc417cf2


### Deco Skills
#### Source
natives/STM/GameDesign/Common/Equip/AccessoryData.user.3.json

<details>

```
    "app.user_data.AccessoryData": {
        "Values": [
            {
                "app.user_data.AccessoryData.cData": {
                    "_Index": 0,
                    "_AccessoryId": {
                        "app.EquipDef.ACCESSORY_ID_Serializable": {
                            "_Value": "[130724024]ACC_ID_0000"
                            }
                    },
                    "_Name": "56bff7fd-6ef8-4038-9ce7-006a0fdc88a1",
                    "_Explain": "abb148ea-836c-4a2a-9cb8-5b74ab3c845c",
                    "_AccessoryType": {
                        "app.EquipDef.ACCESSORY_TYPE_Serializable": {
                            "_Value": "[-1638455296]ACC_TYPE_00"
                        }
                    },
                    "_SortId": 101,
                    "_Rare": {
                        "app.ItemDef.RARE_Serializable": {
                            "_Value": "[16]RARE2"
                        }
                    },
                    "_IconColor": "[4]I_PINK",
                    "_Price": 150,
                    "_SlotLevelAcc": {
                        "app.EquipDef.SlotLevel_Serializable": {
                            "_Value": "[1]Lv1"
                        }
                    },
                    "_Skill": [
                        {
                            "app.HunterDef.Skill_Serializable": {
                                "_Value": "[1]HunterSkill_000"
                            }
                        },
                        {
                            "app.HunterDef.Skill_Serializable": {
                                "_Value": "[0]NONE"
                            }
                        }
                    ],
                    "_SkillLevel": [
                        1,
                        0
                    ]
                }
            },
```
</details>

#### df

In [96]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/AccessoryData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw[0].get('app.user_data.AccessoryData').get('_Values')
df = pd.json_normalize(sect)
df.columns = df.columns.str.replace('app.user_data.AccessoryData.cData._','')

#fill names and descriptions
dict_ = dict(zip(df_deco_ids['guid_n'],df_deco_ids['name']))
df['Name'] = df['Name'].replace(dict_)
dict_ = dict(zip(df_deco_ids['guid_d'],df_deco_ids['description']))
df['Explain'] = df['Explain'].replace(dict_)

#deco size
df.rename(columns={'SlotLevelAcc.app.EquipDef.SlotLevel_Serializable._Value':'deco_size'}, inplace=True)
df['deco_size'] = df['deco_size'].str.extract(r'\[(.*?)\]').astype(int)

# Skill
df['skill_1'] = df['Skill'].str[0].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_2'] = df['Skill'].str[1].str['app.HunterDef.Skill_Serializable'].str['_Value']
df = df.drop('Skill', axis=1)

dict_ = dict(zip(df_common_skills['common_hunter_id'], df_common_skills['name']))
df[['skill_1','skill_2']] = df[['skill_1','skill_2']].replace(dict_)

# Skill Level
df['skill_1_lv'] = df['SkillLevel'].str[0].astype(int)
df['skill_2_lv'] = df['SkillLevel'].str[1].astype(int)
df = df.drop('SkillLevel', axis=1)

# Rarity Clean

# Accessory Type Clean
dict_ = {'[-1638455296]ACC_TYPE_00':'Weapon',
         '[1842954880]ACC_TYPE_01':'Armor'}
df['AccessoryType.app.EquipDef.ACCESSORY_TYPE_Serializable._Value'] = df['AccessoryType.app.EquipDef.ACCESSORY_TYPE_Serializable._Value'].replace(dict_)

#rename
df.rename(columns={'AccessoryId.app.EquipDef.ACCESSORY_ID_Serializable._Value':'deco_id',
                   'AccessoryType.app.EquipDef.ACCESSORY_TYPE_Serializable._Value':'deco_type',
                   'Rare.app.ItemDef.RARE_Serializable._Value':'rarity',
                   'Name':'name',
                   'Explain':'description'},inplace=True)

df_decos = df[['deco_id','name','deco_type','rarity','deco_size','skill_1','skill_1_lv','skill_2','skill_2_lv']].copy()
df_decos.reset_index(drop=True, inplace=True)

# df_decos.head()
df_decos.to_excel('wilds_decos.xlsx')

### Deco Other
#### Source

natives/STM/GameDesign/Common/Equip/AccessoryJudgeData.user.3.json

<details>

```
    "app.user_data.AccessoryJudgeData": {
        "_Values": [
            {
                "app.user_data.AccessoryJudgeData.cData": {
                    "_Index": 0,
                    "_AccessoryId": {
                        "app.EquipDef.ACCESSORY_ID_Serializable": {
                            "_Value": "[130724024]ACC_ID_0000"
                        }
                    },
                    "_Probability": [
                        1,
                        0,
                        0,
                        0,
                        0
                    ]
                }
            },
```
</details>

natives/STM/GameDesign/Common/Equip/AccessoryRankJudgeData.user.3.json

<details>

```
    "app.user_data.AccessoryRankJudgeData": {
        "_Values": [
            {
                "app.user_data.AccessoryRankJudgeData.cData": {
                    "_Index": 0,
                    "_ItemId": {
                        "app.ItemDef.ID_Serializable": {
                            "_Value": "[619]ITEM_0645"
                        }
                    },
                    "_AccessoryType": {
                        "app.EquipDef.ACCESSORY_TYPE_Serializable": {
                            "_Value": "[-1638455296]ACC_TYPE_00"
                        }
                    },
                    "_Probability": [
                        100,
                        0,
                        0,
                        0,
                        0
                    ]
                }
            },
```
</details>

natives/STM/GameDesign/Text/Excel_Equip/AccessoryData.msg.23.json

<details>

```
"entries": [
    {
      "name": "AccessoryData_ACC_EXP_130724024",
      "guid": "443d9dba-6ecc-4c09-9540-6bc5307d186e",
      "crc?": 3624618213,
      "hash": 4289862584,
      "attributes": [],
      "content": [
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        ""
      ]
    },
    {
      "name": "AccessoryData_ACC_130724024",
      "guid": "4f347e66-0474-402f-9bb3-f3c1516fdb91",
      "crc?": 3865111821,
      "hash": 669220903,
      "attributes": [],
      "content": [
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        "",
        ""
      ]
    },
```

</details>

In [97]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/AccessoryJudgeData.user.3.json'

url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/AccessoryRankJudgeData.user.3.json'

url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/AccessoryData.msg.23.json'

## Amulets
### Amulet IDs
#### Source
natives/STM/GameDesign/Text/Excel_Equip/Amulet.msg.23.json
<details>

```
"entries": [
    "name": Amulet_34
    "content": Defense Charm II
    "name": Amulet_EXP34
    "content": A charm that grants the Defense Boost skill
```
</details>

#### df

In [98]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/Amulet.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_2['name'] = df_2['name'].str.replace('EXP','')
df_2.rename(columns={'name':'id','content':'description','guid':'guid_d'}, inplace=True)
df_1.rename(columns={'name':'id','content':'name','guid':'guid_n'}, inplace=True)

#adjust descriptions
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)

#merge into final
df_amulet_ids = pd.merge(df_1[['id','name','guid_n']], df_2[['id','description','guid_d']], on='id', how='left')
df_amulet_ids.reset_index(drop=True, inplace=True)

df_amulet_ids.head()

,id,name,guid_n,description,guid_d
0,Amulet_1,Marathon Charm I,e33f2711-f784-4030-92d8-062905b76265,A charm that grants the Marathon Runner skill.,e3413820-fe5d-4397-8024-4fe212902582
1,Amulet_2,Marathon Charm II,bc7cbc9c-8288-4b6a-a237-7efc0a7bc013,A charm that grants the Marathon Runner skill.,c9785786-2d48-4a7b-a184-11761fd32962
2,Amulet_3,Marathon Charm III,b760fced-112e-4028-b9c6-247f73c2bd9a,A charm that grants the Marathon Runner skill.,e3eb9098-1f35-427b-b848-ad0e3709b5cc
3,Amulet_4,Fitness Charm I,557be166-ff64-4219-aa8f-b5decf7da225,A charm that grants the Constitution skill.,85560730-3118-4eff-bf09-be2f01728569
4,Amulet_5,Fitness Charm II,3419f639-f057-413f-b0b4-43680aa70ed3,A charm that grants the Constitution skill.,5b8ed7c2-7b8e-4402-9e95-a00e7a261351


### Amulet Other
#### Source

natives/STM/GameDesign/Common/Equip/AmuletData.user.3.json

<details>

```
    "app.user_data.AmuletData": {
        "_Values": [
            {
                "app.user_data.AmuletData.cData": {
                    "_Index": 0,
                    "_DataId": 1,
                    "_Lv": 1,
                    "_AmuletType": {
                        "app.ArmorDef.AmuletType_Serializable": {
                            "_Value": "[886754368]AT_0000"
                        }
                    },
                    "_ModelId": 1,
                    "_AmuletColor": {
                        "app.ArmorDef.AmuletColorType_Serializable": {
                            "_Value": "[-1310802816]AMYLET_COLOR_TYPE_00"
                        }
                    },
                    "_NeedEffect": false,
                    "_Name": "e33f2711-f784-4030-92d8-062905b76265",
                    "_Explain": "e3413820-fe5d-4397-8024-4fe212902582",
                    "_SortId": 88,
                    "_Rare": {
                        "app.ItemDef.RARE_Serializable": {
                            "_Value": "[15]RARE3"
                        }
                    },
                    "_IconColor": "[0]I_NONE",
                    "_Price": 1500,
                    "_Skill": [
                        {
                            "app.HunterDef.Skill_Serializable": {
                                "_Value": "[-682586176]HunterSkill_024"
                            }
                        },
                        {
                            "app.HunterDef.Skill_Serializable": {
                                "_Value": "[0]NONE"
                            }
                        },
                        {
                            "app.HunterDef.Skill_Serializable": {
                                "_Value": "[0]NONE"
                            }
                        }
                    ],
                    "_SkillLevel": [
                        1,
                        0,
                        0
                    ]
                }
            },
```
</details>

natives/STM/GameDesign/Catalog/00_00/Data/AmuletColorData.user.3.json

<details>

```
    "app.user_data.AmuletColorData.cEntry": {
        "_Key": "[2]SKILL_0001",
        "STRUCT__Value__Rgba": {
            "x": 0.42352942,
            "y": 0.19607843,
            "z": 0.87058824,
            "w": 1.0
        }
```
</details>

In [99]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/AmuletData.user.3.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw[0].get('app.user_data.AmuletData').get('_Values')
df = pd.json_normalize(sect)
df.columns = df.columns.str.replace('app.user_data.AmuletData.cData._','')

# name and description
dict_ = dict(zip(df_amulet_ids['guid_n'],df_amulet_ids['name']))
df['Name'] = df['Name'].replace(dict_)
dict_ = dict(zip(df_amulet_ids['guid_d'],df_amulet_ids['description']))
df['Explain'] = df['Explain'].replace(dict_)

# skills
df['skill_1'] = df['Skill'].str[0].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_2'] = df['Skill'].str[1].str['app.HunterDef.Skill_Serializable'].str['_Value']
df['skill_3'] = df['Skill'].str[2].str['app.HunterDef.Skill_Serializable'].str['_Value']
df = df.drop('Skill', axis=1)

dict_ = dict(zip(df_common_skills['common_hunter_id'], df_common_skills['name']))
df[['skill_1','skill_2','skill_3']] = df[['skill_1','skill_2','skill_3']].replace(dict_)

# Skill Level
df['skill_1_lv'] = df['SkillLevel'].str[0].astype(int)
df['skill_2_lv'] = df['SkillLevel'].str[1].astype(int)
df['skill_3_lv'] = df['SkillLevel'].str[2].astype(int)
df = df.drop('SkillLevel', axis=1)

#rename
df.rename(columns={'AmuletType.app.ArmorDef.AmuletType_Serializable._Value':'amulet_id',
                   'Rare.app.ItemDef.RARE_Serializable._Value':'rarity',
                   'Name':'name',
                   'Explain':'description'},inplace=True)

df_amulet = df[['amulet_id','name','description','Lv','NeedEffect','rarity','skill_1','skill_1_lv','skill_2','skill_2_lv','skill_3','skill_3_lv']].copy()

df_amulet.head()
# df_amulet.to_excel('wilds charms.xlsx')

,amulet_id,name,description,Lv,NeedEffect,rarity,skill_1,skill_1_lv,skill_2,skill_2_lv,skill_3,skill_3_lv
0,[886754368]AT_0000,Marathon Charm I,A charm that grants the Marathon Runner skill.,1,False,[15]RARE3,Marathon Runner,1,Skill_1,0,Skill_1,0
1,[886754368]AT_0000,Marathon Charm II,A charm that grants the Marathon Runner skill.,2,False,[14]RARE4,Marathon Runner,2,Skill_1,0,Skill_1,0
2,[886754368]AT_0000,Marathon Charm III,A charm that grants the Marathon Runner skill.,3,False,[13]RARE5,Marathon Runner,3,Skill_1,0,Skill_1,0
3,[1744488448]AT_0003,Fitness Charm I,A charm that grants the Constitution skill.,1,False,[15]RARE3,Constitution,1,Skill_1,0,Skill_1,0
4,[1744488448]AT_0003,Fitness Charm II,A charm that grants the Constitution skill.,2,False,[14]RARE4,Constitution,2,Skill_1,0,Skill_1,0


## Weapons

### Weapon Text
#### Source


In [100]:
head = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/'
dict_ = {'bow':'Bow.msg.23.json',
         'bowgun_custom':'BowgunCustomize.msg.23.json',
         'cb':'ChargeAxe.msg.23.json',
         'gl':'GunLance.msg.23.json',
         'ham':'Hammer.msg.23.json',
         'hbg':'HeavyBowgun.msg.23.json',
         'lan':'Lance.msg.23.json',
         'lbg':'LightBowgun.msg.23.json',
         'gs':'LongSword.msg.23.json',
         'ig':'Rod.msg.23.json',
         'kinsect':'RodInsect.msg.23.json',
         'sns':'ShortSword.msg.23.json',
         'swaxe':'SlashAxe.msg.23.json',
         'ls':'Tachi.msg.23.json',
         'db':'TwinSword.msg.23.json',
         'series':'WeaponSeries.msg.23.json',
         'hh':'Whistle.msg.23.json'}
dict_df_weap_ids = {}

for weap,file in dict_.items():
    url = head + file
    with urllib.request.urlopen(url) as page:
        raw = json.load(page)
    sect = raw.get('entries')
    df = pd.json_normalize(sect)

    # pull english names
    df['content'] = df['content'].str[1]

    #separate names and descriptions
    df_1 = df[~df['name'].str.contains('EXP')].copy()
    df_2 = df[df['name'].str.contains('EXP')].copy()

    #normalize naming
    df_2['name'] = df_2['name'].str.replace('EXP','')
    df_2.rename(columns={'name':'id','content':'description','guid':'guid_d'}, inplace=True)
    df_1.rename(columns={'name':'id','content':'name','guid':'guid_n'}, inplace=True)

    #adjust descriptions
    df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)

    #merge into final
    df = pd.merge(df_1[['id','name','guid_n']], df_2[['id','description','guid_d']], on='id', how='left')
    df.reset_index(drop=True, inplace=True)

    #shove into dict
    dict_df_weap_ids[weap]=df.copy()

dict_df_weap_ids['hh'].head()

,id,name,guid_n,description,guid_d
0,Whistle_1,Hope Horn I,88a76352-1055-4e69-8650-6dbb641795ee,A hunting horn made specifically for the exped...,56733819-8573-4ccb-8835-fbae2e48a625
1,Whistle_2,Hope Horn II,300a4735-c0c4-4662-8c2e-a80e02b405ec,A hunting horn made specifically for the exped...,06bda6d7-5e7d-42be-bf7f-e17dac875e13
2,Whistle_3,Hope Horn III,9a24fea5-ccfc-4d69-9c6b-83cb3908b545,A hunting horn made specifically for the exped...,0ec26983-ec1f-4b01-ae61-50717e402130
3,Whistle_4,Hope Horn IV,bda74d0a-9f0c-4a6e-8022-c06703bf83f6,A hunting horn made specifically for the exped...,ee50ecd8-d328-4983-afd9-284e3772566a
4,Whistle_5,Hope Horn V,e6eb8aa5-bae5-42b5-b9aa-0fc33ea2e2b6,A hunting horn made specifically for the exped...,062f67b7-c2ff-4b1e-8a17-96f258a70b75


### Weapon Data

In [111]:
head = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Weapon/'
dict_ = {'bow':'Bow.user.3.json',
         'cb':'ChargeAxe.user.3.json',
         'gl':'GunLance.user.3.json',
         'ham':'Hammer.user.3.json',
         'hbg':'HeavyBowgun.user.3.json',
         'lan':'Lance.user.3.json',
         'lbg':'LightBowgun.user.3.json',
         'gs':'LongSword.user.3.json',
         'ig':'Rod.user.3.json',
         'sns':'ShortSword.user.3.json',
         'swaxe':'SlashAxe.user.3.json',
         'ls':'Tachi.user.3.json',
         'db':'TwinSword.user.3.json',
         'hh':'Whistle.user.3.json'}
dict_df_weap_data = {}

for weap,file in dict_.items():
    url = head + file
    with urllib.request.urlopen(url) as page:
        raw = json.load(page)
    sect = raw[0].get('app.user_data.WeaponData').get('_Values')
    df = pd.json_normalize(sect)
    df.columns = df.columns.str.replace('app.user_data.WeaponData.cData._','')

    # rename proper weapon column
    df_1 = ['LongSword','ShortSword','TwinSword','Tachi','Hammer','Whistle','Lance','GunLance','SlashAxe','ChargeAxe','Rod','Bow','HeavyBowgun','LightBowgun']
    df_2 = df.loc[0, df_1]
    mask = ~df_2.str.contains('INVALID')
    df_1 = df_2.index[mask]
    # if len(df_1) == 1:
    df = df.rename(columns={df_1[0]: 'id'})

    # fill name & detail
    dict_ = dict(zip(dict_df_weap_ids[weap]['guid_n'], dict_df_weap_ids[weap]['name']))
    df['Name'] = df['Name'].replace(dict_)
    dict_ = dict(zip(dict_df_weap_ids[weap]['guid_d'], dict_df_weap_ids[weap]['description']))
    df['Explain'] = df['Explain'].replace(dict_)

    # skill digestion
    df['skill_1'] = df['Skill'].str[0].str['app.HunterDef.Skill_Serializable'].str['_Value']
    df['skill_2'] = df['Skill'].str[1].str['app.HunterDef.Skill_Serializable'].str['_Value']
    df['skill_3'] = df['Skill'].str[2].str['app.HunterDef.Skill_Serializable'].str['_Value']
    df['skill_4'] = df['Skill'].str[3].str['app.HunterDef.Skill_Serializable'].str['_Value']
    df = df.drop('Skill', axis=1)

    dict_ = dict(zip(df_common_skills['common_hunter_id'], df_common_skills['name']))
    df[['skill_1','skill_2','skill_3','skill_4']] = df[['skill_1','skill_2','skill_3','skill_4']].replace(dict_)

    df['skill_1_lv'] = df['SkillLevel'].str[0].astype(int)
    df['skill_2_lv'] = df['SkillLevel'].str[1].astype(int)
    df['skill_3_lv'] = df['SkillLevel'].str[2].astype(int)
    df['skill_4_lv'] = df['SkillLevel'].str[3].astype(int)
    df = df.drop('SkillLevel', axis=1)

    # Slot digestion
    df['slot_1_lv'] = df['SlotLevel'].str[0].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
    df['slot_2_lv'] = df['SlotLevel'].str[1].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
    df['slot_3_lv'] = df['SlotLevel'].str[2].str['app.EquipDef.SlotLevel_Serializable'].str['_Value']
    df = df.drop('SlotLevel', axis=1)

    df['slot_1_lv'] = df['slot_1_lv'].str.extract(r'\[(.*?)\]').astype(int)
    df['slot_2_lv'] = df['slot_2_lv'].str.extract(r'\[(.*?)\]').astype(int)
    df['slot_3_lv'] = df['slot_3_lv'].str.extract(r'\[(.*?)\]').astype(int)

    # Sharpness digestion
    df_1 = ['red','orange','yellow','green','blue','white','purple']
    for i,e in enumerate(df_1):
        df[e] = df['SharpnessValList'].str[i].astype(int)
    df = df.drop('SharpnessValList', axis=1)

    # Attribute digestion



    df.rename(columns={'Rare.app.ItemDef.RARE_Serializable._Value':'rarity',
                       'Critical':'affinity',
                       'Attribute.app.WeaponDef.ATTR_Serializable._Value':'attribute_type',
                       'SubAttribute.app.WeaponDef.ATTR_Serializable._Value':'subattribute_type',
                       'RodInsectLv.app.WeaponDef.ROD_INSECT_LV_Serializable._Value':'kinsect_lv',
                       'Wp08BinType':'swaxe_phial',
                       'Wp08BinValue':'swaxe_phial_val'}, inplace=True)
    
    df = df[['id', 'Name',
            #  'Explain',
             'rarity',
             'Attack',
             'Defense',
             'affinity',
             'attribute_type',
             'AttributeValue',
             'subattribute_type',
             'SubAttributeValue',
             'slot_1_lv',
             'slot_2_lv',
             'slot_3_lv',
             'kinsect_lv',
             'swaxe_phial',
             'swaxe_phial_val', 
             'Wp09BinType',
             'Wp05UniqueType.app.Wp05Def.UNIQUE_TYPE_Serializable._Value',
             'Wp05MusicSkillHighFreqType.app.Wp05Def.WP05_MUSIC_SKILL_HIGH_FREQ_TYPE_Serializable._Value',
             'Wp05HibikiSkillType.app.Wp05Def.WP05_HIBIKI_SKILL_TYPE_Serializable._Value',
             'MainShell.app.MainShellType.TYPE_Serializable._Value',
             'ShellLv',
             'ShellNum',
             'RapidShellNum',
             'IsRappid',
             'CustomizePattern',
             'Wp13SpecialAmmo.app.Wp13Def.SPECIAL_AMMO_TYPE_Serializable._Value',
             'EnergyEfficiency.app.EnergyEfficiencyType.TYPE_Serializable._Value',
             'AmmoStrength.app.AmmoStrengthType.TYPE_Serializable._Value',
             'EnergyShellTypeNormal.app.BowgunEnergyShellType.NORMAL_Serializable._Value',
             'EnergyShellTypePower.app.BowgunEnergyShellType.POWER_Serializable._Value',
             'EnergyShellTypeWeak.app.BowgunEnergyShellType.WEAK_Serializable._Value',
            #  'DispSilencer',
            #  'DispBarrel',
            #  'isLoadingBin',
            #  'SharpnessValList',
             'red',
             'orange',
             'yellow',
             'green',
             'blue',
             'white',
             'purple',
             'TakumiValList',
             'Wp07ShellType.app.Wp07Def.SHELL_TYPE_Serializable._Value',
             'Wp07ShellLv.app.Wp07ShellLevel.SHELL_LV_Serializable._Value',
             'skill_1',
             'skill_1_lv',
             'skill_2',
             'skill_2_lv',
             'skill_3',
             'skill_3_lv',
             'skill_4',
             'skill_4_lv'
             ]]



    dict_df_weap_data[weap] = df.copy()

# dict_df_weap_data['ls']

In [ ]:
dict_df_weap_ids['lbg']

In [112]:
dict_df_weap_data['lbg']

,id,Name,rarity,Attack,Defense,affinity,attribute_type,AttributeValue,subattribute_type,SubAttributeValue,...,Wp07ShellType.app.Wp07Def.SHELL_TYPE_Serializable._Value,Wp07ShellLv.app.Wp07ShellLevel.SHELL_LV_Serializable._Value,skill_1,skill_1_lv,skill_2,skill_2_lv,skill_3,skill_3_lv,skill_4,skill_4_lv
0,[1]LightBowgun_000,Hope Rifle I,[18]RARE0,90,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Skill_1,0,Skill_1,0,Skill_1,0,Skill_1,0
1,[2]LightBowgun_001,Hope Rifle II,[18]RARE0,100,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Ballistics,1,Skill_1,0,Skill_1,0,Skill_1,0
2,[3]LightBowgun_002,Hope Rifle III,[16]RARE2,130,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Ballistics,1,Critical Eye,1,Skill_1,0,Skill_1,0
3,[4]LightBowgun_003,Hope Rifle IV,[14]RARE4,160,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Ballistics,2,Critical Eye,1,Skill_1,0,Skill_1,0
4,[5]LightBowgun_004,Hope Rifle V,[12]RARE6,190,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Ballistics,3,Critical Eye,1,Skill_1,0,Skill_1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,[73]LightBowgun_169,Esperanza Rifle,[11]RARE7,210,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Ballistics,3,Critical Eye,1,Skill_1,0,Skill_1,0
69,[74]LightBowgun_170,Faithbreaker Leibolkule,[11]RARE7,240,20,-5,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Opening Shot,3,Skill_1,0,Skill_1,0,Skill_1,0
70,[75]LightBowgun_171,Expiating Caius,[11]RARE7,200,0,0,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Opening Shot,3,Skill_1,0,Skill_1,0,Skill_1,0
71,[76]LightBowgun_172,Desolate Mist I,[13]RARE5,180,0,15,[0]NONE,0,[0]NONE,0,...,[-324406336]NORMAL,[1226920576]LV1,Slicked Blade,2,Critical Element,1,Skill_1,0,Skill_1,0


In [ ]:
leftovers:
'bow':'BowTree.user.3.json','cb':'ChargeAxeTree.user.3.json','gl':'GunLanceTree.user.3.json','ham':'HammerTree.user.3.json','hbg':'HeavyBowgunTree.user.3.json','lan':'LanceTree.user.3.json','lbg':'LightBowgunTree.user.3.json','gs':'LongSwordTree.user.3.json','ig':'RodTree.user.3.json','sns':'ShortSwordTree.user.3.json','swaxe':'SlashAxeTree.user.3.json','ls':'TachiTree.user.3.json','db':'TwinSwordTree.user.3.json','hh':'WhistleTree.user.3.json','kinsect':'RodInsectData.user.3.json'


### Weapon
#### Source
natives/STM/GameDesign/Common/Equip/WeaponSeriesData.user.3.json

<details>

```
"app.user_data.WeaponSeriesData": {
    "_Values": [
        {
            "app.user_data.WeaponSeriesData.cData": {
                "_Index": 6,
                "_Series": {
                    "app.WeaponDef.SERIES_Serializable": {
                        "_Value": "[-772890752]ID_007"
                    }
                },
                "_Name": "49e1628e-a0ab-4301-a519-73819e97290e"
            }
```
</details>

In [ ]:
https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/BowgunCustomizeItemData.user.3.json
https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/BowgunCustomizeData.user.3.json
https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Common/Equip/WeaponSeriesData.user.3.json


## Misc
### Mantles
#### Source

natives/STM/GameDesign/Text/Excel_Equip/HunterActiveSkill.msg.23.json
<details>

```
"entries": [
    "name": "HunterActiveSkill_3",
    "content": [
        "Continually restores health for\r\na period of time.",
```
</details>

Mantle names:
- chained to item ids in natives/STM/GameDesign/Common/Equip/HunterActiveSkillData.user.3.json
- chained to raw name in natives/STM/GameDesign/Common/Item/itemData.user.3.json
- chained to content in natives/STM/GameDesign/Text/Excel_Data/Item.msg.23.json

#### DF

In [ ]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/HunterActiveSkill.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

# rename columns
df.rename(columns={'name':'id','content':'description'}, inplace=True)

# adjust descriptions
df['description'] = df['description'].str.replace(r'[\r\n]+', ' ', regex=True)


df_mantles = df[['id','description','guid']].copy()

df_mantles.head()

### Meal Skills
#### Source
natives/STM/GameDesign/Text/Excel_Equip/MealSkill.msg.23.json
<details>

```
"entries": [
    "name": "MealSkill_NAME_6",
    "content": [
        "Black Belt Meal (Lo)",
    "name": "MealSkill_EXP_6",
    "content": [
        "回避などの固定でスタミナを\r\n消費する行動のスタミナ消費量が\r\n軽減される。",
        "Reduces stamina depletion\r\nwhen performing stamina-draining\r\nmoves such as evading, etc.",
```
</details>

#### DF

In [ ]:
url = 'https://raw.githubusercontent.com/dtlnor/MHWs-in-json/refs/heads/main/natives/STM/GameDesign/Text/Excel_Equip/MealSkill.msg.23.json'

with urllib.request.urlopen(url) as page:
    raw = json.load(page)
sect = raw.get('entries')
df = pd.json_normalize(sect)

# pull english names
df['content'] = df['content'].str[1]

#separate names and descriptions
df_1 = df[~df['name'].str.contains('EXP')].copy()
df_2 = df[df['name'].str.contains('EXP')].copy()

#normalize naming
df_1['name'] = df_1['name'].str.replace('NAME_','')
df_2['name'] = df_2['name'].str.replace('EXP_','')
df_1.rename(columns={'name':'id','content':'name'}, inplace=True)
df_2.rename(columns={'name':'id','content':'description'}, inplace=True)

#adjust descriptions
df_2['description'] = df_2['description'].str.replace(r'[\r\n]+', ' ', regex=True)

#merge into final
df_meal_skills = pd.merge(df_1[['id','name']], df_2[['id','description']], on='id', how='left')
df_meal_skills.reset_index(drop=True, inplace=True)

df_meal_skills.head()

### NG Words
#### Source
natives/STM/GameDesign/Text/NGWord/
- NGWordDataForMS.user.3.json
- NGWordDataForName.user.3.json
- NGWordDefault.user.3.json

sheet theft

In [ ]:
# import pandas as pd
# url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQwtJ4Y6LJXQ9ORiNE_gTNQuMEQwabLS0lGeKTLPXb4WYHXJwLKDndC-cBl_KCBWQ/pubhtml#"
# df = pd.read_html(url, encoding="utf-8")
# df = df[0]
# df.columns = df.iloc[0]
# df = df[1:].reset_index(drop=True)

# df['Skill_ID'] = df['skillId']
# df['Skill'] = df['skillName']
# df['Skill_Type'] = df['skillCategory']
# df_skills = df[['Index','Skill_ID','Skill','Skill_Type']].copy()

# dict_skills = dict(zip(df_skills['Skill_ID'], df_skills['Skill']))
# # df.head()
# # df.to_excel("Skill_Names.xlsx")

# df = pd.read_excel("mhwilds/SkillData.user.3.xlsx")
# df['skillName'] = df['skillId'].replace(dict_skills)
# df.to_excel('mhwilds/SkillData.user.3.2.xlsx', index=False)